In [ ]:
# Demands numpy 1.25
# !pip install numpy==1.25

In [ ]:
import mne
import numpy as np
from tqdm import tqdm
import scipy.io

In [ ]:
# For each subject => save eeg (filtered 8-30), eog, events, event dictionary  

subject_id_ls = [i for i in range(1, 10)]
# subject_id_ls = [1]

# for dataset in ['T', 'E']: 
for dataset in ['E']: 
    for subject_id in tqdm(subject_id_ls):
        folder = 'train' if dataset == 'T' else 'eval'

        raw = mne.io.read_raw_gdf(f'./data/{folder}/A0{subject_id}{dataset}.gdf', preload=True)

        eog_channels = [ch for ch in raw.ch_names if 'EOG' in ch]

        raw.set_channel_types({ch: 'eog' for ch in eog_channels})

        raw = raw.filter(8, 30, picks='eeg')

        eeg_data = raw.get_data(picks='eeg').T 

        eog_data = raw.get_data(picks='eog').T 

        events, event_id = mne.events_from_annotations(raw)

        if dataset == 'T':
            event_dict = {
                'left_hand':    event_id['769'], 
                'right_hand':   event_id['770'], 
                'feet':         event_id['771'], 
                'tongue':       event_id['772']
                }
        else:
            event_dict = {
                'unknown_cue':  event_id['783'] 
                }
            
        epochs = mne.Epochs(
                raw, 
                events, 
                event_id=event_dict,
                tmin=-0.5, tmax=2.5,
                baseline=None, 
                preload=True,
            )
        
        X = epochs.get_data(picks='eeg') # Shape (n_trials, n_channels, n_samples)
        y = epochs.events[:, 2] # The event IDs (e.g., 769, 770...)
        
        npz_filename = f'./data/{folder}/A0{subject_id}{dataset}.npz'

        raw = mne.io.read_raw_gdf(f'./data/{folder}/A0{subject_id}{dataset}.gdf', preload=True)
        epochs_left = mne.Epochs(
                raw.pick_channels(ch_names=['EEG-0', 'EEG-1', 'EEG-5', 'EEG-C3', 'EEG-6', 'EEG-9', 'EEG-10', 'EEG-14']),
                events, 
                event_id=event_dict,
                tmin=-0.5, tmax=2.5,
                baseline=None, 
                preload=True,
            )

        raw = mne.io.read_raw_gdf(f'./data/{folder}/A0{subject_id}{dataset}.gdf', preload=True)
        epochs_right = mne.Epochs(
                raw.pick_channels(ch_names=['EEG-3', 'EEG-4', 'EEG-7', 'EEG-C4', 'EEG-8', 'EEG-12', 'EEG-13', 'EEG-15']),
                events, 
                event_id=event_dict,
                tmin=-0.5, tmax=2.5,
                baseline=None, 
                preload=True,
            )

        raw = mne.io.read_raw_gdf(f'./data/{folder}/A0{subject_id}{dataset}.gdf', preload=True)
        epochs_central = mne.Epochs(
                raw.pick_channels(ch_names=['EEG-Fz', 'EEG-2', 'EEG-Cz', 'EEG-11', 'EEG-Pz', 'EEG-16']),
                events, 
                event_id=event_dict,
                tmin=-0.5, tmax=2.5,
                baseline=None, 
                preload=True,
            )

        X_left = epochs_left.get_data(picks='eeg') 
        X_right = epochs_right.get_data(picks='eeg')
        X_central = epochs_central.get_data(picks='eeg')

        if dataset == 'T':
            np.savez_compressed(npz_filename, eeg=eeg_data, eog=eog_data, events=events, event_id=event_id, X=X, X_left=X_left, X_right=X_right, X_central=X_central, y=y)
        
        else:
            label_dict = scipy.io.loadmat(f'./data/true_labels/A0{subject_id}E.mat')
            y_true = label_dict['classlabel']
            np.savez_compressed(npz_filename, eeg=eeg_data, eog=eog_data, events=events, event_id=event_id, X=X, X_left=X_left, X_right=X_right, X_central=X_central, y_true=y_true)


  0%|          | 0/9 [00:00<?, ?it/s]

Extracting EDF parameters from /home/dyxcvi/Documents/UCM/experiments/exp#1/data/eval/A01E.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Reading 0 ... 686999  =      0.000 ...  2747.996 secs...


/usr/lib/python3.11/contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 8.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 7.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 413 samples (1.652 s)

Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '783']
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 288 events and 751 original time points ...
0 bad epochs dropped
Extracting EDF parameters from /home/dyxcvi/Documents/UCM/experiments/exp#1/data/train/A01T.gdf...
GDF file detected
Setting channel info structure..

/usr/lib/python3.11/contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


...

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 288 events and 751 original time points ...
0 bad epochs dropped


 22%|██▏       | 2/9 [00:23<01:23, 11.94s/it]

Extracting EDF parameters from /home/dyxcvi/Documents/UCM/experiments/exp#1/data/eval/A03E.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Reading 0 ... 648774  =      0.000 ...  2595.096 secs...


/usr/lib/python3.11/contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)
 22%|██▏       | 2/9 [00:23<01:23, 11.98s/it]


KeyboardInterrupt: 

In [47]:
raw.ch_names

['EEG-0', 'EEG-1', 'EEG-5', 'EEG-C3', 'EEG-6', 'EEG-9', 'EEG-10', 'EEG-14']

In [49]:
ch_names = ['EEG-Fz', 'EEG-0', 'EEG-1', 'EEG-2', 'EEG-3', 'EEG-4', 'EEG-5', 
            'EEG-C3', 'EEG-6', 'EEG-Cz', 'EEG-7', 'EEG-C4', 'EEG-8', 'EEG-9', 
            'EEG-10', 'EEG-11', 'EEG-12', 'EEG-13', 'EEG-14', 'EEG-Pz', 
            'EEG-15', 'EEG-16']

raw = mne.io.read_raw_gdf(f'./data/train/A01T.gdf', preload=True)
epochs_left = mne.Epochs(
        raw.pick_channels(ch_names=['EEG-0', 'EEG-1', 'EEG-5', 'EEG-C3', 'EEG-6', 'EEG-9', 'EEG-10', 'EEG-14']),
        events, 
        event_id=event_dict,
        tmin=-0.5, tmax=2.5,
        baseline=None, 
        preload=True,
    )

raw = mne.io.read_raw_gdf(f'./data/train/A01T.gdf', preload=True)
epochs_right = mne.Epochs(
        raw.pick_channels(ch_names=['EEG-3', 'EEG-4', 'EEG-7', 'EEG-C4', 'EEG-8', 'EEG-12', 'EEG-13', 'EEG-15']),
        events, 
        event_id=event_dict,
        tmin=-0.5, tmax=2.5,
        baseline=None, 
        preload=True,
    )

raw = mne.io.read_raw_gdf(f'./data/train/A01T.gdf', preload=True)
epochs_central = mne.Epochs(
        raw.pick_channels(ch_names=['EEG-Fz', 'EEG-2', 'EEG-Cz', 'EEG-11', 'EEG-Pz', 'EEG-16']),
        events, 
        event_id=event_dict,
        tmin=-0.5, tmax=2.5,
        baseline=None, 
        preload=True,
    )

epochs_left, epochs_right, epochs_central

Extracting EDF parameters from /home/dyxcvi/Documents/UCM/experiments/exp#1/data/train/A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Reading 0 ... 672527  =      0.000 ...  2690.108 secs...


/usr/lib/python3.11/contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 288 events and 751 original time points ...
0 bad epochs dropped
Extracting EDF parameters from /home/dyxcvi/Documents/UCM/experiments/exp#1/data/train/A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Reading 0 ... 672527  =      0.000 ...  2690.108 secs...


/usr/lib/python3.11/contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 288 events and 751 original time points ...
0 bad epochs dropped
Extracting EDF parameters from /home/dyxcvi/Documents/UCM/experiments/exp#1/data/train/A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Reading 0 ... 672527  =      0.000 ...  2690.108 secs...


/usr/lib/python3.11/contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
Not setting metadata
288 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 288 events and 751 original time points ...
0 bad epochs dropped


(<Epochs | 288 events (all good), -0.5 – 2.5 s (baseline off), ~13.2 MiB, data loaded,
  'left_hand': 72
  'right_hand': 72
  'feet': 72
  'tongue': 72>,
 <Epochs | 288 events (all good), -0.5 – 2.5 s (baseline off), ~13.2 MiB, data loaded,
  'left_hand': 72
  'right_hand': 72
  'feet': 72
  'tongue': 72>,
 <Epochs | 288 events (all good), -0.5 – 2.5 s (baseline off), ~9.9 MiB, data loaded,
  'left_hand': 72
  'right_hand': 72
  'feet': 72
  'tongue': 72>)